In [83]:
import serial
import time
import numpy as np

#pyqtgraph -> fast plotting
import pyqtgraph as pg
from pyqtgraph.Qt import QtGui
%gui qt

import copy
from scipy.io import loadmat

In [84]:
#load relevant header data
rawHeaderData = (loadmat('C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Data\\rawframeHeader.mat'))['rawframeHeaderSave'][0]
headerData = (loadmat('C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Data\\frameHeader.mat'))['frameHeaderSave'][0]

In [85]:
#preprocessing
headerStream = np.array([])
for number in rawHeaderData[0]:
    headerStream = np.uint8(np.append(headerStream,number[0]))

In [86]:
def readHeader(recieveHeader):
    headerContent = dict()
    index = 0
    
    headerContent['magicBytes'] = recieveHeader[index:index+8]
    index += 20
    
    headerContent['packetLength'] = recieveHeader[index:index+4].view(dtype=np.uint32)
    index += 4
        
    headerContent['frameNumber'] = recieveHeader[index:index+4].view(dtype=np.uint32)
    index += 24
    
    headerContent['numTLVs'] = recieveHeader[index:index+2].view(dtype=np.uint16)
    
    return headerContent

In [87]:
header = readHeader(headerStream)

In [88]:
header

{'magicBytes': array([2, 1, 4, 3, 6, 5, 8, 7], dtype=uint8),
 'packetLength': array([885], dtype=uint32),
 'frameNumber': array([3670], dtype=uint32),
 'numTLVs': array([3], dtype=uint16)}

In [102]:
dataLength = int(header['packetLength'] - 52)

In [111]:
rawPData = (loadmat('C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Data\\rawPData.mat'))['rawPDataSave'][0][0]

In [ ]:
16
8

In [97]:
pointCloud = np.reshape(p,(4, int(numberOfPoints)))
           
        


In [ ]:
tlvDataLength = tlvLength - tlvHeaderLengthInBytes